In [1]:
import os

In [2]:
%pwd

'c:\\Users\\rahul\\Desktop\\Project\\TextSummarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\rahul\\Desktop\\Project\\TextSummarizer'

Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: str
    gradient_accumulation_steps: int

Configuration Manager

In [6]:
from textSummarizer.constant import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

Components

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

In [9]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = model_pegasus)

        dataset_samsum_features = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir = self.config.root_dir, num_train_epochs = self.config.num_train_epochs, warmup_steps = self.config.warmup_steps,
            per_device_train_batch_size = self.config.per_device_train_batch_size, per_device_eval_batch_size = self.config.per_device_train_batch_size,
            weight_decay = self.config.weight_decay, logging_steps = self.config.logging_steps,
            eval_strategy = self.config.evaluation_strategy, eval_steps = self.config.eval_steps, save_steps = self.config.save_steps,
            gradient_accumulation_steps = self.config.gradient_accumulation_steps
        )

        trainer = Trainer(
            model = model_pegasus,
            args = trainer_args,
            processing_class = tokenizer,
            data_collator = seq2seq_data_collator,
            train_dataset = dataset_samsum_features["validation"],
            eval_dataset = dataset_samsum_features["test"]
        )

        trainer.train()

        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))

Pipeline

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2024-12-15 20:27:07,845: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-15 20:27:07,848: INFO: common: yaml file: params.yaml loaded successfully]


c:\Users\rahul\Desktop\Project\TextSummarizer\.venv\Lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 10%|▉         | 10/102 [03:08<27:43, 18.08s/it]

{'loss': 3.1299, 'grad_norm': 221.40399169921875, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.2}


 20%|█▉        | 20/102 [06:18<26:02, 19.05s/it]

{'loss': 3.1677, 'grad_norm': 856.9864501953125, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.39}


...

{'loss': 2.4214, 'grad_norm': 107.33904266357422, 'learning_rate': 9e-06, 'epoch': 1.74}


 98%|█████████▊| 100/102 [31:42<00:37, 18.63s/it]

{'loss': 2.4761, 'grad_norm': 143.2281036376953, 'learning_rate': 1e-05, 'epoch': 1.94}


100%|██████████| 102/102 [32:17<00:00, 18.52s/it]c:\Users\rahul\Desktop\Project\TextSummarizer\.venv\Lib\site-packages\transformers\modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
100%|██████████| 102/102 [32:27<00:00, 19.09s/it]


{'train_runtime': 1947.3995, 'train_samples_per_second': 0.84, 'train_steps_per_second': 0.052, 'train_loss': 2.7786492039175594, 'epoch': 1.98}
